In [ ]:
import sys
import boto3
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue import DynamicFrame



In [ ]:
# --- Configuration ---
# Ensure that these configuration values are managed securely in a production environment.
glue_db = "airline_cleaned_data_db"  # Glue database name where source tables reside
redshift_cluster_id = "my-redshift-cluster"  # Redshift cluster identifier (ensure correct IAM role permissions)
redshift_db = "dev"  # Redshift database name
redshift_user = "awsuser"  # Redshift username
redshift_schema = "public"  # Redshift schema to store the tables (can be customized)
region = "ap-south-1"  # AWS region where the services are hosted



In [ ]:
# Initialize boto3 clients for Glue and Redshift Data APIs
glue = boto3.client("glue", region_name=region)
redshift = boto3.client("redshift-data", region_name=region)



In [ ]:
# Map Glue data types to Redshift data types for proper table creation
def map_glue_to_redshift(glue_type):
    """
    Maps AWS Glue data types to Redshift compatible data types for table creation.
    """
    mapping = {
        "string": "VARCHAR(256)",
        "int": "INTEGER",
        "bigint": "BIGINT",
        "double": "FLOAT8",
        "float": "FLOAT4",
        "boolean": "BOOLEAN",
        "timestamp": "TIMESTAMP",
        "date": "DATE"
    }
    return mapping.get(glue_type.lower(), "VARCHAR(256)")  # Default to VARCHAR for unsupported types

# Read data from Glue catalog into dynamic frames
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Reading the tables from the Glue catalog; this assumes Glue Catalog is properly configured
ad_arrivals_node = glueContext.create_dynamic_frame.from_catalog(database=glue_db, table_name="ad_arrivals", transformation_ctx="ad_arrivals_node")
ad_flights_node = glueContext.create_dynamic_frame.from_catalog(database=glue_db, table_name="ad_flights", transformation_ctx="ad_flights_node")
ad_departures_node = glueContext.create_dynamic_frame.from_catalog(database=glue_db, table_name="ad_departures", transformation_ctx="ad_departures_node")



In [ ]:
# Function to write dynamic frames to Redshift, including table creation logic
def write_to_redshift(dynamic_frame, table_name):
    """
    Writes a Glue DynamicFrame to Redshift.
    - Generates DDL based on the dynamic frame's schema.
    - Creates the table in Redshift if it does not already exist.
    - Loads the data into Redshift after table creation.
    """
    # Extract the schema from the dynamic frame
    columns = dynamic_frame.schema().fields

    # Generate CREATE TABLE DDL
    ddl = f"CREATE TABLE IF NOT EXISTS {redshift_schema}.{table_name} (\n"
    ddl += ",\n".join([
        f"  {col.name} {map_glue_to_redshift(col.dataType.simpleString())}"  # Map Glue types to Redshift
        for col in columns
    ])
    ddl += "\n);"

    # Execute the CREATE TABLE statement in Redshift
    try:
        result = redshift.execute_statement(
            ClusterIdentifier=redshift_cluster_id,
            Database=redshift_db,
            DbUser=redshift_user,
            Sql=ddl
        )
        print(f"Table {table_name} created successfully in Redshift.")
    except Exception as e:
        # Log error to CloudWatch for production troubleshooting
        print(f"Failed to create table '{table_name}': {e}")
        # In production, consider sending errors to a monitoring system (e.g., CloudWatch, SNS, etc.)
        raise  # Re-raise exception to ensure job fails and triggers alerts

    # Load the dynamic frame into the newly created Redshift table
    try:
        glueContext.write_dynamic_frame.from_options(
            frame=dynamic_frame,
            connection_type="redshift",
            connection_options={
                "redshiftTmpDir": "s3://aws-glue-assets-681451696920-ap-south-1/temporary/",  # Temporary directory for intermediate data
                "useConnectionProperties": "true",  # Use Glue connection properties
                "dbtable": f"{redshift_schema}.{table_name}",
                "connectionName": "AirlineRedshiftConnection"  # Ensure Redshift connection is properly configured in Glue Catalog
            },
            transformation_ctx=f"write_{table_name}_to_redshift"
        )
        print(f"Data loaded into Redshift table {table_name}.")
    except Exception as e:
        # Log error to CloudWatch for production troubleshooting
        print(f"Failed to load data into Redshift table '{table_name}': {e}")
        raise  # Re-raise exception to ensure job fails

# Write the dynamic frames to Redshift (production-level)
# In production, consider logging the time it takes to load the data and monitoring resource usage (e.g., CPU, memory)

write_to_redshift(ad_arrivals_node, "ad_arrivals")
write_to_redshift(ad_flights_node, "ad_flights")
write_to_redshift(ad_departures_node, "ad_departures")

# Mark the Glue job as complete
job.commit()


